# This notebook is used to look at individual player performances

In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector


from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from sklearn.dummy import DummyRegressor

from sklearn.metrics import plot_roc_curve

from xgboost import XGBRegressor
from sklearn.decomposition import PCA

from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.svm import SVR

In [49]:
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

# Tom Brady 

These stats were gathered for Tom Brady using the 

In [12]:
df = pd.read_csv('./tb_stats.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 61 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           83 non-null     float64
 1   Year         83 non-null     int64  
 2   Date         83 non-null     object 
 3   G#           83 non-null     float64
 4   Week         83 non-null     float64
 5   Age          83 non-null     float64
 6   Tm           83 non-null     object 
 7   Home         83 non-null     float64
 8   Opp          83 non-null     object 
 9   Cmp          83 non-null     float64
 10  Att          83 non-null     float64
 11  Cmp%         83 non-null     float64
 12  Yds          83 non-null     float64
 13  TD_left      83 non-null     float64
 14  Int          83 non-null     float64
 15  Rate         83 non-null     float64
 16  Sk           83 non-null     float64
 17  Sk_Yds       83 non-null     float64
 18  Y/A          83 non-null     float64
 19  AY/A      

In [14]:
df.head()

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,RUSH YDS,RUSH ATT,RUSH TD,TD_right,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
0,273.0,2017,2017-09-07,1.0,1.0,40.035,NWE,1.0,KAN,16.0,...,1890,443,15,39,51,28,54.9,352,40.1,52.9
1,274.0,2017,2017-09-17,2.0,2.0,40.045,NWE,0.0,NOR,30.0,...,1787,406,11,36,48,25,52.1,308,41.0,33.3
2,275.0,2017,2017-09-24,3.0,3.0,40.052,NWE,1.0,HOU,25.0,...,1747,440,14,48,54,33,61.1,301,35.3,62.5
3,276.0,2017,2017-10-01,4.0,4.0,40.059,NWE,1.0,CAR,32.0,...,1409,353,7,34,44,21,47.7,282,37.9,35.0
4,277.0,2017,2017-10-05,5.0,5.0,40.063,NWE,0.0,TAM,30.0,...,1880,440,18,44,53,29,54.7,327,48.1,22.2


In [15]:
from pandas_profiling import ProfileReport

In [16]:
profile = ProfileReport(df, title="QBs")

In [17]:
#profile.to_notebook_iframe()

In [18]:
df.corr()

,Rk,Year,G#,Week,Age,Home,Cmp,Att,Cmp%,Yds,...,RUSH YDS,RUSH ATT,RUSH TD,TD_right,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
Rk,1.000000,0.980590,0.135077,0.134844,0.985092,-0.046498,0.208034,0.218235,0.054054,0.070734,...,-0.108005,-0.165325,0.052792,-0.067936,-0.014056,0.063924,0.062106,-0.109885,0.085507,0.230744
Year,0.980590,1.000000,-0.061693,-0.061698,0.999699,-0.055236,0.236149,0.242484,0.064499,0.091371,...,-0.161226,-0.215983,-0.013860,-0.123106,-0.063719,0.012621,0.027217,-0.159172,0.093306,0.220464
G#,0.135077,-0.061693,1.000000,0.998630,-0.037241,0.043728,-0.126658,-0.105949,-0.048957,-0.093605,...,0.288906,0.278139,0.355925,0.298930,0.277307,0.284140,0.189121,0.273857,-0.019220,0.069353
Week,0.134844,-0.061698,0.998630,1.000000,-0.037221,0.038845,-0.134845,-0.115244,-0.050050,-0.100059,...,0.286071,0.276143,0.353815,0.300122,0.276797,0.282864,0.187354,0.270207,-0.024762,0.064446
Age,0.985092,0.999699,-0.037241,-0.037221,1.000000,-0.054377,0.233168,0.239951,0.063439,0.089154,...,-0.154346,-0.209406,-0.005062,-0.115800,-0.056814,0.019770,0.031969,-0.152582,0.092977,0.222458
Home,-0.046498,-0.055236,0.043728,0.038845,-0.054377,1.000000,-0.096654,-0.097390,-0.033416,-0.047588,...,0.049819,0.062670,-0.016843,-0.028553,0.016639,0.003549,0.018192,0.049679,-0.193663,0.145605
Cmp,0.208034,0.236149,-0.126658,-0.134845,0.233168,-0.096654,1.000000,0.807666,0.606008,0.767032,...,-0.029138,0.041605,0.121216,0.032495,0.117018,0.079056,-0.017212,0.091688,0.214352,0.002511
Att,0.218235,0.242484,-0.105949,-0.115244,0.239951,-0.097390,0.807666,1.000000,0.030870,0.558316,...,-0.025004,0.026854,0.000637,-0.049268,0.060606,-0.001761,-0.084492,0.049291,0.123413,-0.108816
Cmp%,0.054054,0.064499,-0.048957,-0.050050,0.063439,-0.033416,0.606008,0.030870,1.000000,0.561411,...,-0.020199,0.032362,0.199296,0.116312,0.115350,0.135190,0.087613,0.096173,0.186627,0.154311
Yds,0.070734,0.091371,-0.093605,-0.100059,0.089154,-0.047588,0.767032,0.558316,0.561411,1.000000,...,0.166622,0.169893,0.248432,0.216943,0.233722,0.230792,0.109590,0.247586,0.269464,0.112492


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 61 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           83 non-null     float64
 1   Year         83 non-null     int64  
 2   Date         83 non-null     object 
 3   G#           83 non-null     float64
 4   Week         83 non-null     float64
 5   Age          83 non-null     float64
 6   Tm           83 non-null     object 
 7   Home         83 non-null     float64
 8   Opp          83 non-null     object 
 9   Cmp          83 non-null     float64
 10  Att          83 non-null     float64
 11  Cmp%         83 non-null     float64
 12  Yds          83 non-null     float64
 13  TD_left      83 non-null     float64
 14  Int          83 non-null     float64
 15  Rate         83 non-null     float64
 16  Sk           83 non-null     float64
 17  Sk_Yds       83 non-null     float64
 18  Y/A          83 non-null     float64
 19  AY/A      

In [22]:
X = df.drop(['Total','Date', 'Tm', 'Re_Y/R', 'Re_Ctch%', 'Re_Y/Tgt'], axis=1)
y = df['Total']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=.2)

In [30]:
#subpipes that scale numeric data and use one hot encoder on categorical 
subpipe_num = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('ss', StandardScaler())
])

subpipe_cat = Pipeline(steps=[
    ('cat_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [26]:
#cat_feat = ['Player', 'Team', 'Opp', 'Day']

In [31]:
CT = ColumnTransformer(transformers=[
    ('subpipe_num', subpipe_num, selector(dtype_include=np.number)),
     ('subpipe_cat', subpipe_cat, selector(dtype_include=object))], remainder='passthrough')

In [33]:
X_train.shape

(66, 55)

In [34]:
#Create a pipeline for dummy model
dummy_model_pipe = Pipeline(steps=[
    ('ct', CT),
    ('dum', DummyRegressor(strategy='median'))
])

In [35]:
dummy_model_pipe.fit(X_train, y_train)
dummy_model_pipe.score(X_train,y_train)

-0.010661927004583127

In [36]:
X_train

,Rk,Year,G#,Week,Age,Home,Opp,Cmp,Att,Cmp%,...,RUSH YDS,RUSH ATT,RUSH TD,TD_right,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
67,340.0,2021,4.0,4.0,44.061,0.0,NWE,22.0,43.0,51.16,...,2103,464,9,35,48,23,47.9,308,36.5,53.1
56,328.0,2020,8.0,8.0,43.091,0.0,NYG,28.0,40.0,70.00,...,1782,432,14,37,59,30,50.8,353,44.6,61.1
40,312.0,2019,8.0,8.0,42.085,1.0,CLE,20.0,36.0,55.56,...,2315,463,19,46,57,35,61.4,345,37.9,35.3
9,281.0,2017,9.0,10.0,40.101,0.0,DEN,25.0,34.0,73.53,...,1430,428,9,44,41,24,58.5,266,31.6,33.3
61,336.0,2020,16.0,17.0,43.153,1.0,ATL,26.0,41.0,63.41,...,1677,380,15,49,56,36,64.3,367,41.0,43.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,293.0,2018,5.0,5.0,41.062,1.0,IND,34.0,44.0,77.27,...,1626,415,12,37,43,23,53.5,315,41.0,46.7
60,334.0,2020,14.0,15.0,43.139,0.0,ATL,31.0,45.0,68.89,...,1677,380,15,49,56,36,64.3,367,41.0,43.5
71,344.0,2021,8.0,8.0,44.089,0.0,NOR,28.0,40.0,70.00,...,1589,426,12,35,46,20,43.5,304,37.1,42.9
14,286.0,2017,14.0,15.0,40.136,0.0,PIT,22.0,35.0,62.86,...,1693,385,14,36,39,24,61.5,270,36.2,31.6


# Simple Regression Models

In [37]:
regressors = [
    
    LinearRegression(),
    Ridge(),
    Lasso(),
    SVR(),
]

In [44]:
for regressor in regressors:
    steps = [
        ('ct', CT),
        ('rg', regressor)
    ]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)   
    print(regressor)
    print("model score: %.3f" % pipeline.score(X_test, y_test))
    

LinearRegression()
model score: 1.000
Ridge()
model score: 0.995
Lasso()
model score: 0.882
SVR()
model score: 0.006


In [50]:
import xgboost as xgb

In [51]:
GB = xgb.XGBRegressor(random_state=42, max_depth=4, n_estimators=100)

In [52]:
xgb_model_pipe = Pipeline(steps=[
    ('ct', CT),
    ('xgb', GB)
])

In [53]:
xgb_model_pipe.fit(X_train, y_train)
xgb_model_pipe.score(X_train,y_train)

0.9999999951970597

In [54]:
xgb_model_pipe.score(X_test,y_test)

0.772922067366111

In [48]:
GB.get_booster().get_score(importance_type='gain')

{'f66': 394.49909728727283,
 'f4': 49.48782446925227,
 'f0': 32.77533458432616,
 'f7': 95.7566926228182,
 'f120': 108.0439642,
 'f17': 106.89743047416665,
 'f116': 117.87890103333332,
 'f139': 153.315918,
 'f119': 58.44504729,
 'f86': 31.365922056,
 'f18': 80.54382482749999,
 'f90': 227.4006612714286,
 'f138': 302.35917179999996,
 'f53': 242.19234423777777,
 'f91': 240.1742121141667,
 'f10': 94.47490656310528,
 'f9': 64.01977525695654,
 'f40': 186.08434643333337,
 'f6': 52.74347865137364,
 'f1': 85.21952926511959,
 'f3': 52.656180856960006,
 'f152': 13.80699062,
 'f161': 100.91599275,
 'f70': 251.10892278666668,
 'f2': 45.36051557522641,
 'f114': 52.3935547,
 'f108': 100.860588,
 'f46': 119.4721825875,
 'f29': 97.45327227400001,
 'f137': 177.47263665,
 'f75': 263.22943291999997,
 'f169': 123.57600858399996,
 'f5': 55.885211533279055,
 'f11': 82.54917195242103,
 'f74': 254.3821975566667,
 'f16': 79.23280558942858,
 'f13': 91.40847152347825,
 'f20': 257.29550504428573,
 'f146': 100.63993

In [50]:
GB.feature_importances_

array([0.00270671, 0.00703774, 0.00374604, 0.00434854, 0.00408689,
       0.0046152 , 0.00435575, 0.00790794, 0.00400935, 0.00528699,
       0.00780208, 0.00681721, 0.00372377, 0.00754885, 0.0059825 ,
       0.00831518, 0.00654333, 0.00882798, 0.0066516 , 0.00558778,
       0.0212484 , 0.00153521, 0.01504395, 0.00386501, 0.00130628,
       0.00466605, 0.01013859, 0.00491964, 0.        , 0.00804805,
       0.0059165 , 0.00612711, 0.00375284, 0.        , 0.00741539,
       0.00854506, 0.00076292, 0.00612917, 0.00333445, 0.00934843,
       0.01536753, 0.00808752, 0.00422838, 0.0030429 , 0.01194427,
       0.00416214, 0.00986645, 0.00459925, 0.00347844, 0.0029101 ,
       0.        , 0.00261506, 0.00262955, 0.02000113, 0.00121297,
       0.        , 0.00847739, 0.00110433, 0.00234278, 0.00876121,
       0.00119912, 0.00760778, 0.0092751 , 0.00337462, 0.00303831,
       0.00288882, 0.03257918, 0.00264285, 0.00092532, 0.00224983,
       0.02073749, 0.00206265, 0.0044855 , 0.        , 0.02100

In [44]:
transformer_pipe = Pipeline(steps=[
    ('ct', CT)
])

In [45]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1011 entries, 32 to 1126
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    1011 non-null   object 
 1   Day       1011 non-null   object 
 2   Week      1011 non-null   int64  
 3   Age       1011 non-null   int64  
 4   Team      1011 non-null   object 
 5   Home      1011 non-null   int64  
 6   Opp       1011 non-null   object 
 7   Dome      1011 non-null   int64  
 8   RANK      1011 non-null   int64  
 9   PTS       1011 non-null   int64  
 10  SACKS     1011 non-null   int64  
 11  INT       1011 non-null   int64  
 12  TO        1011 non-null   int64  
 13  PTS.1     1011 non-null   int64  
 14  PASS YDS  1011 non-null   int64  
 15  PASS TD   1011 non-null   int64  
 16  RUSH YDS  1011 non-null   int64  
 17  RUSH TD   1011 non-null   int64  
 18  RZ ATT    1011 non-null   int64  
 19  RZ TD     1011 non-null   int64  
 20  RZ TD%    1011 non-null   flo

In [62]:
ohe = OneHotEncoder()

In [65]:
X_train_ohe = ohe.fit_transform(X_train[['Day', 'Team', 'Opp']])

In [66]:
X_test_ohe = ohe.transform(X_test[['Day', 'Team', 'Opp']])

# Neural Network 

In [67]:
X_train_ohe

<1011x71 sparse matrix of type '<class 'numpy.float64'>'
	with 3033 stored elements in Compressed Sparse Row format>

In [82]:
X_train

,Day,Week,Age,Team,Home,Opp,Dome,RANK,PTS,SACKS,...,PASS YDS,PASS TD,RUSH YDS,RUSH TD,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
32,Sun,16,27,WAS,1,CAR,0,18,18,29,...,3825,28,1936,17,57,36,63.2,360,49.2,60.0
156,Sun,5,23,ARI,0,NYJ,1,26,8,31,...,4409,34,1792,16,60,36,60.0,381,44.6,41.7
382,Thu,1,24,KAN,1,HOU,1,27,6,34,...,4104,30,2564,24,63,40,63.5,390,47.5,52.4
1166,Sun,8,44,TAM,0,NOR,1,4,12,46,...,3821,20,1589,12,46,20,43.5,304,37.1,42.9
365,Thu,5,31,CHI,1,TAM,0,9,8,48,...,3945,29,1289,10,51,32,62.7,319,40.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,Sun,17,23,BAL,1,LAR,1,15,0,49,...,4109,17,1754,18,56,29,51.8,347,41.3,53.1
1095,Thu,11,24,ATL,1,NWE,1,2,18,36,...,3181,21,2103,9,48,23,47.9,308,36.5,53.1
1130,Sun,10,36,NYJ,1,BUF,0,1,8,41,...,2771,12,1866,19,45,23,51.1,285,30.8,45.7
860,Sun,16,33,LAR,0,MIN,2,24,12,51,...,4300,29,2222,15,59,33,55.9,391,36.4,50.0


In [ ]:
def scale_datasets(X_train_ohe, X_test_ohe):
    standard_scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        standard_scaler.fit_transform(X_train_ohe),
        columns=X_train_ohe.columns
  )
    X_test_scaled = pd.DataFrame(
        standard_scaler.transform(X_test_ohe),
        columns = X_test_ohe.columns
  )
    return X_train_scaled, X_test_scaled
X_train_scaled, X_test_scaled = scale_datasets(X_train_ohe, X_test_ohe)

In [85]:
hidden_units1 = 32
hidden_units2 = 64
hidden_units3 = 128
learning_rate = 0.01
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
    model = Sequential([
        Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
        Dense(1, kernel_initializer='normal', activation='linear')
      ])
    return model
# build the model
model = build_model_using_sequential()

In [88]:
# loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=['accuracy']
)
# train the model
history = model.fit(
    X_train_CT, 
    y_train, 
    epochs=1000, 
    batch_size=64,
    validation_split=0.2
)

Epoch 1/1000
13/13 [==============================] - 0s 8ms/step - loss: 0.2164 - accuracy: 0.0025 - val_loss: 1.1506 - val_accuracy: 0.0000e+00
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2060 - accuracy: 0.0037 - val_loss: 1.1959 - val_accuracy: 0.0000e+00
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2006 - accuracy: 0.0025 - val_loss: 1.2168 - val_accuracy: 0.0000e+00
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1865 - accuracy: 0.0037 - val_loss: 1.1928 - val_accuracy: 0.0000e+00
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1831 - accuracy: 0.0037 - val_loss: 1.2050 - val_accuracy: 0.0000e+00
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1925 - accuracy: 0.0025 - val_loss: 1.1598 - val_accuracy: 0.0000e+00
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1942 - accuracy: 0.0025 - val_loss: 1.1387 - val

Epoch 57/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1415 - accuracy: 0.0012 - val_loss: 1.3457 - val_accuracy: 0.0000e+00
Epoch 58/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1301 - accuracy: 0.0025 - val_loss: 1.2927 - val_accuracy: 0.0000e+00
Epoch 59/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1454 - accuracy: 0.0037 - val_loss: 1.2453 - val_accuracy: 0.0000e+00
Epoch 60/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1478 - accuracy: 0.0037 - val_loss: 1.2008 - val_accuracy: 0.0000e+00
Epoch 61/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1522 - accuracy: 0.0025 - val_loss: 1.3151 - val_accuracy: 0.0000e+00
Epoch 62/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1492 - accuracy: 0.0037 - val_loss: 1.2393 - val_accuracy: 0.0000e+00
Epoch 63/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1668 - accuracy: 0.0025 - val_loss: 1.193

13/13 [==============================] - 0s 2ms/step - loss: 0.1259 - accuracy: 0.0012 - val_loss: 1.1967 - val_accuracy: 0.0000e+00
Epoch 168/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1055 - accuracy: 0.0037 - val_loss: 1.2376 - val_accuracy: 0.0000e+00
Epoch 169/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1229 - accuracy: 0.0025 - val_loss: 1.2163 - val_accuracy: 0.0000e+00
Epoch 170/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1165 - accuracy: 0.0037 - val_loss: 1.2216 - val_accuracy: 0.0000e+00
Epoch 171/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1148 - accuracy: 0.0025 - val_loss: 1.2402 - val_accuracy: 0.0000e+00
Epoch 172/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1272 - accuracy: 0.0037 - val_loss: 1.1917 - val_accuracy: 0.0000e+00
Epoch 173/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1155 - accuracy: 0.0037 - val_loss: 1.2389 - val_

13/13 [==============================] - 0s 2ms/step - loss: 0.0816 - accuracy: 0.0037 - val_loss: 1.2169 - val_accuracy: 0.0000e+00
Epoch 278/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0914 - accuracy: 0.0025 - val_loss: 1.2032 - val_accuracy: 0.0000e+00
Epoch 279/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0916 - accuracy: 0.0037 - val_loss: 1.2046 - val_accuracy: 0.0000e+00
Epoch 280/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0979 - accuracy: 0.0025 - val_loss: 1.2479 - val_accuracy: 0.0000e+00
Epoch 281/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0875 - accuracy: 0.0037 - val_loss: 1.2155 - val_accuracy: 0.0000e+00
Epoch 282/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1042 - accuracy: 0.0037 - val_loss: 1.2804 - val_accuracy: 0.0000e+00
Epoch 283/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1131 - accuracy: 0.0025 - val_loss: 1.2376 - val_

13/13 [==============================] - 0s 2ms/step - loss: 0.0963 - accuracy: 0.0012 - val_loss: 1.2185 - val_accuracy: 0.0000e+00
Epoch 388/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0803 - accuracy: 0.0025 - val_loss: 1.1827 - val_accuracy: 0.0000e+00
Epoch 389/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0851 - accuracy: 0.0025 - val_loss: 1.2037 - val_accuracy: 0.0000e+00
Epoch 390/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1076 - accuracy: 0.0037 - val_loss: 1.1363 - val_accuracy: 0.0000e+00
Epoch 391/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0935 - accuracy: 0.0012 - val_loss: 1.1810 - val_accuracy: 0.0000e+00
Epoch 392/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0904 - accuracy: 0.0012 - val_loss: 1.1877 - val_accuracy: 0.0000e+00
Epoch 393/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1056 - accuracy: 0.0025 - val_loss: 1.2044 - val_

13/13 [==============================] - 0s 2ms/step - loss: 0.0917 - accuracy: 0.0037 - val_loss: 1.1924 - val_accuracy: 0.0000e+00
Epoch 498/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0740 - accuracy: 0.0037 - val_loss: 1.1552 - val_accuracy: 0.0000e+00
Epoch 499/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0792 - accuracy: 0.0037 - val_loss: 1.1777 - val_accuracy: 0.0000e+00
Epoch 500/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0873 - accuracy: 0.0037 - val_loss: 1.1642 - val_accuracy: 0.0000e+00
Epoch 501/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0985 - accuracy: 0.0025 - val_loss: 1.2112 - val_accuracy: 0.0000e+00
Epoch 502/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1109 - accuracy: 0.0037 - val_loss: 1.1984 - val_accuracy: 0.0000e+00
Epoch 503/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0885 - accuracy: 0.0025 - val_loss: 1.2271 - val_

13/13 [==============================] - 0s 3ms/step - loss: 0.0700 - accuracy: 0.0025 - val_loss: 1.1423 - val_accuracy: 0.0000e+00
Epoch 608/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0673 - accuracy: 0.0037 - val_loss: 1.1597 - val_accuracy: 0.0000e+00
Epoch 609/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0852 - accuracy: 0.0012 - val_loss: 1.1534 - val_accuracy: 0.0000e+00
Epoch 610/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0711 - accuracy: 0.0025 - val_loss: 1.1554 - val_accuracy: 0.0000e+00
Epoch 611/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0784 - accuracy: 0.0025 - val_loss: 1.1588 - val_accuracy: 0.0000e+00
Epoch 612/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0794 - accuracy: 0.0012 - val_loss: 1.1448 - val_accuracy: 0.0000e+00
Epoch 613/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0726 - accuracy: 0.0037 - val_loss: 1.1623 - val_

13/13 [==============================] - 0s 3ms/step - loss: 0.0825 - accuracy: 0.0025 - val_loss: 1.1973 - val_accuracy: 0.0000e+00
Epoch 718/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0788 - accuracy: 0.0037 - val_loss: 1.1842 - val_accuracy: 0.0000e+00
Epoch 719/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0786 - accuracy: 0.0025 - val_loss: 1.0890 - val_accuracy: 0.0000e+00
Epoch 720/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0868 - accuracy: 0.0025 - val_loss: 1.1253 - val_accuracy: 0.0000e+00
Epoch 721/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1028 - accuracy: 0.0012 - val_loss: 1.1436 - val_accuracy: 0.0000e+00
Epoch 722/1000
13/13 [==============================] - 0s 8ms/step - loss: 0.0735 - accuracy: 0.0025 - val_loss: 1.1950 - val_accuracy: 0.0000e+00
Epoch 723/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0888 - accuracy: 0.0025 - val_loss: 1.1861 - val_

13/13 [==============================] - 0s 3ms/step - loss: 0.0701 - accuracy: 0.0037 - val_loss: 1.2329 - val_accuracy: 0.0000e+00
Epoch 828/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0769 - accuracy: 0.0012 - val_loss: 1.1419 - val_accuracy: 0.0000e+00
Epoch 829/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0881 - accuracy: 0.0037 - val_loss: 1.1391 - val_accuracy: 0.0000e+00
Epoch 830/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0682 - accuracy: 0.0037 - val_loss: 1.1198 - val_accuracy: 0.0000e+00
Epoch 831/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0841 - accuracy: 0.0037 - val_loss: 1.1404 - val_accuracy: 0.0000e+00
Epoch 832/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0613 - accuracy: 0.0037 - val_loss: 1.1775 - val_accuracy: 0.0000e+00
Epoch 833/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0776 - accuracy: 0.0037 - val_loss: 1.1292 - val_

13/13 [==============================] - 0s 2ms/step - loss: 0.0592 - accuracy: 0.0037 - val_loss: 1.1046 - val_accuracy: 0.0000e+00
Epoch 938/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0633 - accuracy: 0.0037 - val_loss: 1.1564 - val_accuracy: 0.0000e+00
Epoch 939/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0604 - accuracy: 0.0037 - val_loss: 1.1973 - val_accuracy: 0.0000e+00
Epoch 940/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0635 - accuracy: 0.0037 - val_loss: 1.1364 - val_accuracy: 0.0000e+00
Epoch 941/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0652 - accuracy: 0.0037 - val_loss: 1.1645 - val_accuracy: 0.0000e+00
Epoch 942/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.0585 - accuracy: 0.0037 - val_loss: 1.1789 - val_accuracy: 0.0000e+00
Epoch 943/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.0661 - accuracy: 0.0037 - val_loss: 1.1460 - val_